In [ ]:
import pandas as pd
import numpy as np

data=pd.read_csv("/content/USA_Housing.csv")


data.head()

Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)  
Download the dataset regarding USA House Price Prediction from the following link:  
https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing  
Load the dataset and Implement 5- fold cross validation for multiple linear regression
(using least square error fit).  
Steps:  
a) Divide the dataset into input features (all columns except price) and output variable  
(price)

In [ ]:
x=data.drop('Price',axis=1)
y=data['Price']

b) Scale the values of input features.  


In [ ]:
import sklearn.preprocessing as preprocessing
x=preprocessing.scale(x)
print(x)

[[ 1.02865969 -0.29692705  0.02127433  0.08806222 -1.31759867]
 [ 1.00080775  0.02590164 -0.25550611 -0.72230146  0.40399945]
 [-0.68462915 -0.11230283  1.5162435   0.93084045  0.07240989]
 ...
 [-0.48723454  1.28447022 -2.17026949 -1.50025059 -0.29193658]
 [-0.05459152 -0.44669439  0.14154061  1.18205319  0.65111608]
 [-0.28831272  0.01521477 -0.19434166  0.07185495  1.04162464]]


c) Divide input and output features into five folds.  

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=42)
print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)



d) Run five iterations, in each iteration consider one-fold as test set and remaining
four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score
for each iteration using least square error fit.  


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

beta_matrices = []
predicted_values = []
r2_scores = []

for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = LinearRegression()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    beta_matrices.append(model.coef_)
    predicted_values.append(y_pred)
    r2_scores.append(r2_score(y_test, y_pred))

    print(f"Fold R2 Score: {r2_scores[-1]}")
    print(f"Fold Beta Matrix: {beta_matrices[-1]}")
    print("-" * 20)
print("")
print("Average R2 Score:", np.mean(r2_scores))

Fold R2 Score: 0.9179971706985147
Fold Beta Matrix: [230745.94073479 163243.27314515 120309.77397759   3011.45976111
 151552.63069359]
--------------------
Fold R2 Score: 0.9145677884802819
Fold Beta Matrix: [229081.97914235 165882.1605634  121536.57475055   2092.4478622
 150874.99274586]
--------------------
Fold R2 Score: 0.9116116385364478
Fold Beta Matrix: [230224.50511001 162766.17455493 121022.77324577   1247.16258975
 150234.77720419]
--------------------
Fold R2 Score: 0.9193091764960816
Fold Beta Matrix: [229500.10043209 165212.07110924 122839.9376815    3063.71699324
 150917.88484984]
--------------------
Fold R2 Score: 0.9243869413350316
Fold Beta Matrix: [230225.0513193  163956.83884606 121115.12045628    783.46716975
 150662.44678192]
--------------------

Average R2 Score: 0.9175745431092714


e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Identify the best beta matrix based on maximum R2 score
best_r2_idx = np.argmax(r2_scores)
best_coefficients = beta_matrices[best_r2_idx]

# Split data into training (70%) and testing (30%)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

# Initialize and train the Linear Regression model
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train, y_train)

# Predict on the test set
y_test_pred = lin_reg_model.predict(X_test)

# Evaluate model performance
test_r2 = r2_score(y_test, y_test_pred)

# Print results
print(f"R² Score (70/30 split): {test_r2:.4f}")
print(f"Model Coefficients (Beta Matrix): {lin_reg_model.coef_}")

R2 Score on 70/30 split using model trained on 70% data: 0.9146818498916267
Beta Matrix from model trained on 70% data: [230464.52520478 164159.19982569 120514.71328324   2913.62424674
 151019.35865135]


**Q2.** Concept of Validation set for Multiple Linear Regression (Gradient Descent  
Optimization).Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the
dataset into training set (56%), validation set (14%), and test set (30%). Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of
regression coefficients for each value of learning rate after 1000 iterations. For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regession coefficient.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load dataset
housing_data = pd.read_csv("USA_Housing.csv")
X_features = housing_data.drop(columns=['Price']).values
y_target = housing_data['Price'].values.reshape(-1, 1)

# Split into train, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X_features, y_target, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.20, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Add bias column
def add_bias_column(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

X_train_bias = add_bias_column(X_train)
X_val_bias = add_bias_column(X_val)
X_test_bias = add_bias_column(X_test)

# Gradient descent implementation
def gradient_descent_solver(X, y, learning_rate, iterations=1000):
    m, n = X.shape
    coefficients = np.zeros((n, 1))
    for _ in range(iterations):
        gradient = (1/m) * (X.T @ (X @ coefficients - y))
        coefficients -= learning_rate * gradient
    return coefficients

# Try multiple learning rates and track best model
learning_rates = [0.001, 0.01, 0.1, 1]
best_r2_val = -np.inf
best_coefficients = None
best_learning_rate = None

for lr in learning_rates:
    coefficients = gradient_descent_solver(X_train_bias, y_train, lr, iterations=1000)
    y_val_pred = X_val_bias @ coefficients
    y_test_pred = X_test_bias @ coefficients

    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Print results for each learning rate (unchanged)
    print(f"Learning Rate = {lr}")
    print("R² on Validation Set:", r2_val)
    print("R² on Test Set:", r2_test)
    print("Beta Coefficients:\n", coefficients.flatten()[:5], "...")
    print("-" * 40)

    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_coefficients = coefficients
        best_learning_rate = lr

# Print best model details (unchanged)
print("\nBest Learning Rate:", best_learning_rate)
print("Best Validation R²:", best_r2_val)
print("Best Beta Coefficients:\n", best_coefficients.flatten())


Learning Rate = 0.001
R² on Validation Set: -0.799334149357984
R² on Test Set: -0.9872590714052005
Beta Coefficients:
 [779274.02090139 146658.52639308 103040.26919133  72303.5523883
  24381.66647824] ...
----------------------------------------
Learning Rate = 0.01
R² on Validation Set: 0.909818561206292
R² on Test Set: 0.9147438859798211
Beta Coefficients:
 [1232381.36998704  234542.29331285  162395.51342821  121485.70570465
    3089.76469737] ...
----------------------------------------
Learning Rate = 0.1
R² on Validation Set: 0.9097995626742029
R² on Test Set: 0.9147570103083724
Beta Coefficients:
 [1232434.57572502  234562.99454568  162415.95445045  121760.31153933
    2814.16200824] ...
----------------------------------------
Learning Rate = 1
R² on Validation Set: 0.9097995626742028
R² on Test Set: 0.9147570103083724
Beta Coefficients:
 [1232434.57572502  234562.99454568  162415.95445045  121760.31153933
    2814.16200824] ...
----------------------------------------

Best Lea

Pre-processing and Multiple Linear Regression  
Download the dataset regarding Car Price Prediction from the following link:  
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data  
1. Load the dataset with following column names ["symboling", "normalized_losses",  
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",  
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",  
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",  
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]  
and replace all ? values with NaN  
2. Replace all NaN values with central tendency imputation. Drop the rows with NaN  
values in price column  
3. There are 10 columns in the dataset with non-numeric values. Convert these values to  
numeric values using following scheme:  
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures  
for e.g., two to 2  
(ii) For "body_style", "drive_wheels": use dummy encoding scheme  
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding  
scheme  
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.  
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.  
4. Divide the dataset into input features (all columns except price) and output variable  
(price). Scale all input features.  
5. Train a linear regressor on 70% of data (using inbuilt linear regression function of  
Python) and test its performance on remaining 30% of data.  
6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then  
again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of python. Does it lead to any perfromance improvement on the test set?

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.decomposition import PCA

# Column names for the dataset
column_names = [
    "symboling", "normalized_losses", "make", "fuel_type", "aspiration",
    "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
    "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
    "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
    "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"
]

# Load dataset
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
car_data = pd.read_csv(dataset_url, names=column_names)

# Replace missing values represented as "?" with NaN
car_data = car_data.replace("?", np.nan)

# Impute missing values: mode for categorical, median for numeric
for col in car_data.columns:
    if car_data[col].dtype == "object":
        mode_val = car_data[col].mode()[0]
        car_data[col] = car_data[col].fillna(mode_val)
    else:
        car_data[col] = pd.to_numeric(car_data[col], errors="coerce")
        median_val = car_data[col].median()
        car_data[col] = car_data[col].fillna(median_val)

# Remove rows with missing price values
car_data = car_data.dropna(subset=["price"])
car_data["price"] = car_data["price"].astype(float)

# Encode categorical values
num_encoding = {"two": 2, "three": 3, "four": 4, "five": 5,
                "six": 6, "eight": 8, "twelve": 12}
car_data["num_doors"] = car_data["num_doors"].replace(num_encoding).astype(int)
car_data["num_cylinders"] = car_data["num_cylinders"].replace(num_encoding).astype(int)

# One-hot encoding for some categorical columns
car_data = pd.get_dummies(car_data, columns=["body_style", "drive_wheels"], drop_first=True)

# Label encoding for selected columns
for col in ["make", "aspiration", "engine_location", "fuel_type"]:
    encoder = LabelEncoder()
    car_data[col] = encoder.fit_transform(car_data[col])

# Binary encoding for engine_type and fuel_system
car_data["fuel_system"] = car_data["fuel_system"].apply(lambda x: 1 if "pfi" in x else 0)
car_data["engine_type"] = car_data["engine_type"].apply(lambda x: 1 if "ohc" in x else 0)

# Split features and target variable
X_features = car_data.drop(columns=["price"]).values
y_target = car_data["price"].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_target, test_size=0.3, random_state=42)

# Linear Regression without PCA
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

r2_original = r2_score(y_test, y_pred)
mse_original = mean_squared_error(y_test, y_pred)

# Print results (unchanged)
print("Linear Regression without PCA")
print("R² Score:", r2_original)
print("MSE:", mse_original)

# PCA for dimensionality reduction
pca_transformer = PCA(n_components=0.95)
X_pca = pca_transformer.fit_transform(X_scaled)

Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_pca, y_target, test_size=0.3, random_state=42)

# Linear Regression with PCA
lr_model_pca = LinearRegression()
lr_model_pca.fit(Xp_train, yp_train)
yp_pred = lr_model_pca.predict(Xp_test)

r2_pca = r2_score(yp_test, yp_pred)
mse_pca = mean_squared_error(yp_test, yp_pred)

# Print results (unchanged)
print("\nLinear Regression with PCA")
print("R² Score:", r2_pca)
print("MSE:", mse_pca)

# Performance comparison
if r2_pca > r2_original:
    print("\n✅ PCA improved performance")
else:
    print("\n❌ PCA did not improve performance")


Linear Regression without PCA
R² Score: 0.7895045576733848
MSE: 14448999.011837844

Linear Regression with PCA
R² Score: 0.7478420860380317
MSE: 17308828.207359694

❌ PCA did not improve performance


/tmp/ipython-input-949230477.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_doors"] = data["num_doors"].replace(num_map).astype(int)
/tmp/ipython-input-949230477.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_cylinders"] = data["num_cylinders"].replace(num_map).astype(int)
